## 3. Realistic simulation setup
- Generate the world points:
    - ~~Generate a spot~~
    - ~~apply the grid~~
    - discretization of the spot
    - Multiple spots
    - Each spot has 1cm diameter, the distance between corners is 2cm.
    - Each spot must have 4 world points.

### import  libraries

In [140]:
vscode = 1

In [141]:
if(vscode == 0):
    # for vscode
    # for jupyter notebook
    from mpl_toolkits.mplot3d import axes3d
    import matplotlib.pyplot as plt

    %matplotlib notebook
elif(vscode == 1):
    %matplotlib qt
    

In [142]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 
from typing import Sequence
from calib_lib import *
DECIMALS = 2            # how many decimal places to use in print


### Generate World spots

In [143]:
F = 16                                       # focal length( in mm )
image_size = np.array([1936,1216])               # sensor size(in mm)
image_size = np.array([11.345,7.126])
pixel_width = 5.86e-3                       # pixel size in mm
PX= image_size[0]/2.0                       # principal point x-coordinate
PY= image_size[1]/2.0                       # principal point y-coordinate
IMAGE_HEIGTH = image_size[1]
IMAGE_WIDTH = image_size[0]
THETA_X = 0                                 # roll angle
THETA_Y = 0                                 # pitch angle
THETA_Z = 0                         # yaw angle


# camera Right
THETA_X_R = 0                                 # roll angle
THETA_Y_R = 0                                 # pitch angle
THETA_Z_R= 0                                 # yaw angle
# camera Left
THETA_X_L = 0                                 # roll angle
THETA_Y_L = 0                                 # pitch angle
THETA_Z_L= 0                                 # yaw angle

C_L = np.array([0,0,0])                     # camera centre
C_R = np.array([500,0,0])

chess_dimx,chess_dimy = (13,9)
chess_sq_size = 44

## Generate spots
Generate spots with 1cm diameter, which equates to 4 spots displaced in a retangular shape.

In [144]:
zdistance = 10000 # 10 meters
spots = np.array([[245,110,zdistance],
         [250,115,zdistance],
         [255,110,zdistance],
         [250,105,zdistance]])
fig = plt.figure(figsize = (20,8))
ax = fig.add_subplot( projection = '3d')
ax.plot(spots[:,0],spots[:,1],spots[:,2], color = 'r',ls = "None", marker = "*",label = "World point w/ 1 cm diameter")


In [145]:
x_arr_L, X_arr_L, E_L,K_L, P_L = get_image_points(spots,PX,PY,thetax=THETA_X_L,thetay = THETA_Y_L,thetaz = THETA_Z_L,trans_x= -C_L[0],trans_y= -C_L[1],trans_z= -C_L[2],F = F)
print(x_arr_L)
x_arr_R, X_arr_R, E_R, K_R,P_R = get_image_points(spots,PX,PY,thetax = THETA_X_R,thetay = THETA_Y_R, thetaz = THETA_Z_R,trans_x= -C_R[0],trans_y= -C_R[1],trans_z= -C_R[2],F = F)
print(x_arr_R)

[[6.0645 3.739 ]
 [6.0725 3.747 ]
 [6.0805 3.739 ]
 [6.0725 3.731 ]]
[[5.2645 3.739 ]
 [5.2725 3.747 ]
 [5.2805 3.739 ]
 [5.2725 3.731 ]]


## Find the circle
Find the circle based on 3 points


In [146]:
def findcircle(x_arr):
    Ax  = x_arr[0,0]
    Ay  = x_arr[0,1]
    Bx  = x_arr[1,0]
    By  = x_arr[1,1]
    Cx  = x_arr[2,0]
    Cy  = x_arr[2,1]
    Dx  = x_arr[3,0]
    Dy  = x_arr[3,1]

    radius = (Cx-Ax)/2
    circle_x = (Ax+Cx)/2
    circle_y = (By+Dy)/2
    return circle_x,circle_y,radius

def createcircle(x_arr,nrpoints = 100):

    circle_x,circle_y,radius = findcircle(x_arr)

    theta = np.linspace(0, 2*np.pi, nrpoints)
    r  = radius
    x = r*np.cos(theta)
    y = r*np.sin(theta)

    return x+circle_x, y+circle_y,radius

In [147]:
circle_x,circle_y,radius = findcircle(x_arr_R)
print("radius: ",radius)

radius:  0.008000000000000007


## Plot of the spot in the image plane

In [148]:
image_size_pixel = np.array([1936,1216])               # sensor size(in mm)
fig = plt.figure(figsize=(10,10))
fig.suptitle('Left and Right image coordinates', fontsize=16)

ax_L_orig = fig.add_subplot(121)
ax_L_orig.plot(x_arr_L[:,0],x_arr_L[:,1], color = 'r',ls = "None", marker = ".", label = 'Original')
ax_L_orig.set_xlabel("X axis")
ax_L_orig.set_ylabel("Y axis")
ax_L_orig.grid()
ax_L_orig.legend()
ax_L_orig.set_xlim([0,image_size[0]])
ax_L_orig.set_ylim([0,image_size[1]])

# circle definition
xL,yL,rL = findcircle(x_arr_L)
cc_L = plt.Circle(( xL ,yL ), rL)
ax_L_orig.add_artist( cc_L ) 
ax_L_orig.set_aspect( 1 ) 

ax_R_orig = fig.add_subplot(122)
ax_R_orig.plot(x_arr_R[:,0],x_arr_R[:,1], color = 'r',ls = "None", marker = ".", label = 'Original')
ax_R_orig.set_xlabel("X axis")
ax_R_orig.set_ylabel("Y axis")
ax_R_orig.grid()
ax_R_orig.legend()
ax_R_orig.set_xlim([0,image_size[0]])
ax_R_orig.set_ylim([0,image_size[1]])

# circle definition
xR,yR,rR = findcircle(x_arr_R)
cc_R = plt.Circle(( xR ,yR ), rR)
ax_R_orig.add_artist( cc_R ) 
ax_R_orig.set_aspect( 1 ) 

plt.show()

## Location of the plot

In [149]:
image_size = np.array([1936,1216])               # sensor size(in mm)
pixelGridX  = (np.arange(0,image_size[0])) * pixel_width
pixelGridY  = (np.arange(0,image_size[1])) * pixel_width
pixelGridXX, pixelGridYY = np.meshgrid(pixelGridX,pixelGridY)
pixelGridXX, pixelGridYY = np.meshgrid(pixelGridX,pixelGridY)

In [150]:
fig = plt.figure(figsize=(20,20))
ax_fig_L = fig.add_subplot(221)
ax_fig_L.plot(pixelGridXX,pixelGridYY,'b.-', label = 'Original')
ax_fig_L.plot(pixelGridXX.T,pixelGridYY.T, 'r.-', label = 'Original')
#circle definition
xL,yL,rL = findcircle(x_arr_L)
cc_L = plt.Circle(( xL ,yL ), rL)
ax_fig_L.add_artist( cc_L ) 
ax_fig_L.set_aspect( 1 ) 
#limits
x_min = xL-(rL*5)
x_max = xL+(rL*5)
y_min = yL-(rL*5)
y_max = yL+(rL*5)
ax_fig_L.set_xlim([x_min,x_max])
ax_fig_L.set_ylim([y_min,y_max])

ax_fig_mini_L = fig.add_subplot(222)

ax_fig_mini_L.plot(pixelGridXX,pixelGridYY,'b.-', label = 'Original')
ax_fig_mini_L.plot(pixelGridXX.T,pixelGridYY.T, 'r.-', label = 'Original')
#circle definition
cc_L = plt.Circle(( xL ,yL ), rL)
ax_fig_mini_L.add_artist( cc_L ) 
ax_fig_mini_L.set_aspect( 1 ) 
#limits
x_min = xL-(rL*2)
x_max = xL+(rL*2)
y_min = yL-(rL*2)
y_max = yL+(rL*2)
ax_fig_mini_L.set_xlim([x_min,x_max])
ax_fig_mini_L.set_ylim([y_min,y_max])

### Right image

ax_fig_R = fig.add_subplot(223)
ax_fig_R.plot(pixelGridXX,pixelGridYY,'b.-', label = 'Original')
ax_fig_R.plot(pixelGridXX.T,pixelGridYY.T, 'r.-', label = 'Original')
#circle definition
xR,yR,rR = findcircle(x_arr_R)
cc_R = plt.Circle(( xR ,yR ), rR)
ax_fig_R.add_artist( cc_R ) 
ax_fig_R.set_aspect( 1 ) 
#limits
x_min = xR-(rR*5)
x_max = xR+(rR*5)
y_min = yR-(rR*5)
y_max = yR+(rR*5)
ax_fig_R.set_xlim([x_min,x_max])
ax_fig_R.set_ylim([y_min,y_max])

ax_fig_mini_R = fig.add_subplot(224)

ax_fig_mini_R.plot(pixelGridXX,pixelGridYY,'b.-', label = 'Original')
ax_fig_mini_R.plot(pixelGridXX.T,pixelGridYY.T, 'r.-', label = 'Original')
cc_R = plt.Circle(( xR ,yR ), rR)
ax_fig_mini_R.add_artist( cc_R ) 
ax_fig_mini_R.set_aspect( 1 ) 
#limits
x_min = xR-(rR*2)
x_max = xR+(rR*2)
y_min = yR-(rR*2)
y_max = yR+(rR*2)
ax_fig_mini_R.set_xlim([x_min,x_max])
ax_fig_mini_R.set_ylim([y_min,y_max])

plt.show()

## Plot spot with pixel diameter
- ~~create a mesh with pixel size distance corners~~
- ~~create a spot centered in the middle of the plane~~
- ~~Pass the spot through the mesh~~
- ~~Expand the mesh by 1 in each direction~~
- Each pixel that has the spot goes to a micro mesh
- Each micropixel that has the spot sum.

In [167]:
# Generate pixel grid
pixelGridX  = (np.arange(0,image_size[0])) * pixel_width
pixelGridY  = (np.arange(0,image_size[1])) * pixel_width
pixelGridXX, pixelGridYY = np.meshgrid(pixelGridX,pixelGridY)

In [168]:
gridSize = 1024
theta = np.linspace(0, 2*np.pi, gridSize)
#center of the spot
cx0 = (image_size[0]/2* pixel_width)
cy0 = (image_size[1]/2* pixel_width)
#radius
r  = pixel_width
x0 = r*np.cos(theta)+(cx0)
y0 = r*np.sin(theta)+(cy0)

fig, ax = plt.subplots()
ax.plot(x0,y0)

ax.set_aspect(1)

plt.show()

In [169]:
fig, ax = plt.subplots()
ax.plot(pixelGridXX,pixelGridYY)
ax.plot(pixelGridXX.T,pixelGridYY.T)
ax.plot(x0,y0, linewidth=4)

ax.set_xlim([cx0-(2*pixel_width),cx0+(2*pixel_width)])
ax.set_ylim([cy0-(2*pixel_width),cy0+(2*pixel_width)])


ax.set_aspect(1)
plt.show()

In [170]:
circ1 = np.sqrt((pixelGridXX-(image_size[0]/2* pixel_width))**2 + (pixelGridYY-(image_size[1]/2* pixel_width))**2) <= r
circ = np.where(circ1 == True)
circ

(array([607, 608, 608], dtype=int64), array([968, 968, 969], dtype=int64))

In [171]:
fig, ax = plt.subplots()
ax.plot(pixelGridXX,pixelGridYY)
ax.plot(pixelGridXX.T,pixelGridYY.T)
ax.plot(x0,y0, linewidth=4)
ax.plot(pixelGridX[circ[1][:]],pixelGridY[circ[0][:]],'r*')


ax.set_xlim([cx0-(2*pixel_width),cx0+(2*pixel_width)])
ax.set_ylim([cy0-(2*pixel_width),cy0+(2*pixel_width)])

ax.set_aspect(1)
plt.show()

In [172]:
step_dist = 2
xx_pixel =  np.linspace(circ[1][:]-step_dist,circ[1][:]+step_dist,(step_dist*2 + 1))
yy_pixel =  np.linspace(circ[0][:]-step_dist,circ[0][:]+step_dist,(step_dist*2 + 1))
xx_pixel

array([[966., 966., 967.],
       [967., 967., 968.],
       [968., 968., 969.],
       [969., 969., 970.],
       [970., 970., 971.]])

In [157]:
len(circ[0][:])

3

In [166]:
fig,ax = plt.subplots()
pixel_orient = np.array([[1,1],[1,-1],[-1,1],[-1,-1]])

ax.plot(pixelGridXX,pixelGridYY)
ax.plot(pixelGridXX.T,pixelGridYY.T)
ax.plot(x0,y0, linewidth=4)
ax.plot(pixelGridX[circ[1][:]],pixelGridY[circ[0][:]],'r*')
for j in range(len(circ[0][:])):
    for i in range(4):
        rectangle = plt.Rectangle((pixelGridX[circ[1][j]],pixelGridY[circ[0][j]]), pixel_orient[i,0]*pixel_width,pixel_orient[i,1]*pixel_width , fc='green',ec="red")
        plt.gca().add_patch(rectangle)
        ax.axis('scaled')

ax.set_xlim([cx0-(2*pixel_width),cx0+(2*pixel_width)])
ax.set_ylim([cy0-(2*pixel_width),cy0+(2*pixel_width)])

ax.set_aspect(1)

plt.show()

In [ ]:
gridSize = 2001
theta = np.linspace(0, 2*np.pi, gridSize)
r  = pixel_width/2
x0 = r*np.cos(theta)
y0 = r*np.sin(theta)

xgrid      = np.linspace(-1,1,gridSize) * pixel_width
xx, yy     = np.meshgrid(xgrid,xgrid)  
pixelColor = np.zeros(xx.shape)

circ1 = np.sqrt((xx-0)**2 + (yy-0)**2) <= r
circ = np.where(circ1 == True)

fig, ax = plt.subplots()
ax.plot(x0,y0)
ax.plot(xx,yy)
ax.plot(xx.T,yy.T)
ax.plot(yy[circ[0][:],0],yy[circ[1][:],0],'r*')

ax.set_aspect(1)
plt.show()